# Decision Trees

Incase of classification, leaf nodes contain label and in case of regression, the predixtion is obtain by taking sample mean of the labels of the subset if training examples present in that leaf node.

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
eps=np.finfo(float).eps
eps

2.220446049250313e-16

Here eps is the smallest representable number. At times we get `log(0)` or `0` in the denominator, to avoid that we are going to use this

## Demo classification

In [ ]:
df = pd.DataFrame([['overcast', 'hot', 'high', 'FALSE', 'yes'],
['overcast', 'cool', 'normal', 'TRUE', 'yes'],
['overcast', 'mild', 'high', 'TRUE', 'yes'],
['overcast', 'hot', 'normal', 'FALSE', 'yes'],
['rainy', 'mild', 'high', 'FALSE', 'yes'],
['rainy', 'cool', 'normal', 'FALSE', 'yes'],
['rainy', 'cool', 'normal', 'TRUE', 'no'],
['rainy', 'mild', 'normal', 'FALSE', 'yes'],
['rainy', 'mild', 'high', 'TRUE', 'no'],
['sunny', 'hot', 'high', 'FALSE', 'no'],
['sunny', 'hot', 'high', 'TRUE', 'no'],
['sunny', 'mild', 'high', 'FALSE', 'no'],
['sunny', 'cool', 'normal', 'FALSE', 'yes'],
['sunny', 'mild', 'normal', 'TRUE', 'yes']], columns=['outlook','temp','humidity','windy','play'])

In [ ]:
df.shape

(14, 5)

In [ ]:
df.values

array([['overcast', 'hot', 'high', 'FALSE', 'yes'],
       ['overcast', 'cool', 'normal', 'TRUE', 'yes'],
       ['overcast', 'mild', 'high', 'TRUE', 'yes'],
       ['overcast', 'hot', 'normal', 'FALSE', 'yes'],
       ['rainy', 'mild', 'high', 'FALSE', 'yes'],
       ['rainy', 'cool', 'normal', 'FALSE', 'yes'],
       ['rainy', 'cool', 'normal', 'TRUE', 'no'],
       ['rainy', 'mild', 'normal', 'FALSE', 'yes'],
       ['rainy', 'mild', 'high', 'TRUE', 'no'],
       ['sunny', 'hot', 'high', 'FALSE', 'no'],
       ['sunny', 'hot', 'high', 'TRUE', 'no'],
       ['sunny', 'mild', 'high', 'FALSE', 'no'],
       ['sunny', 'cool', 'normal', 'FALSE', 'yes'],
       ['sunny', 'mild', 'normal', 'TRUE', 'yes']], dtype=object)

In [ ]:
df.keys()

Index(['outlook', 'temp', 'humidity', 'windy', 'play'], dtype='object')

In [ ]:
target = df.keys()[-1]
target


'play'

In [ ]:
df[target].unique()

array(['yes', 'no'], dtype=object)

In [ ]:
df[target].value_counts()

yes    9
no     5
Name: play, dtype: int64

In [ ]:
df[target].unique()[0]

'yes'

In [ ]:
print(df[target].value_counts()[df[target].unique()[0]])
print(df[target].value_counts()[df[target].unique()[1]])

9
5


## Calculate entropy of whole dataset

In [ ]:
def find_entropy_whole(df):
  target = df.keys()[-1]

  #intialization
  overall_entropy=0

  #possible values of target
  values_in_target = df[target].unique()

  for value in values_in_target:
    p=df[target].value_counts()[value]/len(df[target])
    overall_entropy+=-p*np.log2(p)
  return overall_entropy

find_entropy_whole(df)


0.9402859586706309

## Calculating entropy of an attribute

In [ ]:
def find_entropy_of_attribute(df,attribute):
  #last column in df is the target variable
  target=df.keys()[-1]

  #possible values of target
  values_in_target = df[target].unique()

  values_in_attribute = df[attribute].unique()

  entropy_attribute=0

  for value_in_attribute in values_in_attribute:
    overall_entropy=0
    for value_in_target in values_in_target:
      num= len(df[attribute][df[attribute]==value_in_attribute][df[target]==value_in_target])
      den =len(df[attribute][df[attribute]==value_in_attribute])
      p=num/(den+eps)
      overall_entropy+=-p*np.log2(p+eps)
    p2=den/len(df)
    entropy_attribute+=-p2*overall_entropy
  return abs(entropy_attribute)

In [ ]:
for i_attribute in df.keys()[:-1]:
  print(f'Entropy of attribute \'{i_attribute}\'is: ', find_entropy_of_attribute(df,i_attribute))

Entropy of attribute 'outlook'is:  0.6935361388961914
Entropy of attribute 'temp'is:  0.9110633930116756
Entropy of attribute 'humidity'is:  0.7884504573082889
Entropy of attribute 'windy'is:  0.892158928262361


## Finding best attribute

In [ ]:
def find_best_attribute_to_divide(df):
  #informatio gain
  IG=[]

  #all column names
  all_attribute_names=df.keys()[:-1]

  for attribute in all_attribute_names:
    #compute IG for every attribute
    IG.append(find_entropy_whole(df)-find_entropy_of_attribute(df,attribute))

  #get the index of attribute with best IG
  index_of_attribute_with_max_IG=np.argmax(IG)

  #print(index of attribute with max IG)
  best_attribute = all_attribute_names[index_of_attribute_with_max_IG]
  return best_attribute

find_best_attribute_to_divide(df)

'outlook'

## Building Decision Tree

In [ ]:
def buildTree(df,tree=None):

  #last column  in dataframe
  target = df.keys()[-1]

  #Here we build our decision tree

  #Get attribute with maximum information gain
  node = find_best_attribute_to_divide(df)

  #Get distinct values of that attribute
  attValue = np.unique(df[node])

  #Create empty dictionary to create tree
  if tree is None:
    tree={}
    tree[node]={}
  
  #We make loop to construct a tree by calling this function recursively.
  #In this we check if the subset is pure and stops if it is pure
  for value in attValue:
    subtable = df[df[node]== value].reset_index(drop=True)
    c1Value, counts = np.unique(subtable['play'],return_counts=True)

    if len(counts)==1: #Checking the purity of subset
      tree[node][value]=c1Value[0]
    else:
      tree[node][value]=buildTree(subtable)
  return tree

In [ ]:
buildTree(df)

{'outlook': {'overcast': 'yes',
  'rainy': {'windy': {'FALSE': 'yes', 'TRUE': 'no'}},
  'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}